In [112]:
from __future__ import division
import pandas as pd
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Musical_Instruments.json.gz')
print(df.head(5))
df=df.head(50000)


       reviewerID        asin           reviewerName helpful  \
0  A1YS9MDZP93857  0006428320            John Taylor  [0, 0]   
1  A3TS466QBAWB9D  0014072149          Silver Pencil  [0, 0]   
2  A3BUDYITWUSIS7  0041291905  joyce gabriel cornett  [0, 0]   
3  A19K10Z0D2NTZK  0041913574            TexasCowboy  [0, 0]   
4  A14X336IB4JD89  0201891859                 dfjm53  [0, 1]   

                                          reviewText  overall  \
0  The portfolio is fine except for the fact that...      3.0   
1  If you are a serious violin student on a budge...      5.0   
2  This is and excellent edition and perfectly tr...      5.0   
3  Perfect for someone who is an opera fan or a w...      5.0   
4  How many Nocturnes does it contain? All of the...      1.0   

                            summary  unixReviewTime   reviewTime  
0                     Parts missing      1394496000  03 11, 2014  
1  Perform it with a friend, today!      1370476800   06 6, 2013  
2           Vivalldi's 

In [113]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import os
print(os.listdir())

import warnings
warnings.filterwarnings('ignore')
dataset = pd.read_csv("GFG1.csv")
dataset=dataset.head(50000)
Y=dataset["label"]


['.ipynb_checkpoints', 'algo - B.ipynb', 'archive.zip', 'birnn.ipynb', 'cnn.ipynb', 'GFG1.csv', 'glove.6B.100d.txt', 'mlp.ipynb', 'reviews_Musical_Instruments.json', 'reviews_Musical_Instruments.json.gz', 'Untitled - Copy.ipynb', 'Untitled - Copy_.ipynb']


In [114]:
X = df['reviewText']
y = dataset["label"]
y=y.replace((np.inf, -np.inf, np.NaN), False).reset_index(drop=True)

def pro(pic):
    v=(df.loc[df['reviewText'].str.contains(pic)]) 
    v=(v.index)
    v=(v[0])

    return(Y.iloc[[v]])


In [115]:
from __future__ import print_function
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from bs4 import BeautifulSoup
import logging
from numpy import random
from nltk.corpus import stopwords
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import re
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, MaxPooling1D, Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

In [116]:
def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    #text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    return text
X = X.map(lambda a: clean_text(a))
def text_to_wordlist(text):    
    #Remove Special Characters
    text = re.sub(r'[^a-z\d ]', " ", text)
    text = re.sub(r'\d+', '_num_', text)    
    return(text)
X = X.map(lambda a: text_to_wordlist(a))



In [117]:
train, test, y_train, y_test = train_test_split(X,y,stratify=y,random_state=42)

In [118]:
tokenizer = Tokenizer(num_words=None,lower=True,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',split=' ',char_level=False)
tokenizer.fit_on_texts(X)
x_train = tokenizer.texts_to_sequences(X)
x_test = tokenizer.texts_to_sequences(X)
word_index = tokenizer.word_index
X = tokenizer.texts_to_sequences(X)
vocab_size = len(word_index)
y_train=y
print('Vocab size: {}'.format(vocab_size))
longest = max(len(seq) for seq in X)
print("Longest comment size: {}".format(longest))
average = np.mean([len(seq) for seq in X])
y_test=y
print("Average comment size: {}".format(average))
stdev = np.std([len(seq) for seq in X])
print("Stdev of comment size: {}".format(stdev))
max_len = int(average + stdev * 3)
print('Max comment size: {}'.format(max_len))

Vocab size: 76540
Longest comment size: 3571
Average comment size: 99.84224
Stdev of comment size: 130.81840356686212
Max comment size: 492


In [119]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
#!pip install keras==2.3.1
processed_post_x_train = pad_sequences(x_train, maxlen=max_len, padding='post', truncating='post')
processed_post_x_test = pad_sequences(x_test, maxlen=max_len, padding='post', truncating='post')
processed_x_train = pad_sequences(x_train, maxlen=max_len)
processed_x_test = pad_sequences(x_test, maxlen=max_len)
print('x_train shape:', processed_x_train.shape)
print('x_test shape:', processed_x_test.shape)
import tensorflow.keras.backend
from keras.models import Sequential, load_model
from keras.layers import GRU, Dense, Conv1D, MaxPooling1D
from keras.layers import Dropout, GlobalMaxPooling1D, BatchNormalization, LSTM
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Nadam
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt

x_train shape: (50000, 492)
x_test shape: (50000, 492)


In [120]:
import os
embeddings_index = {}
f = open('glove.6B.100d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))
embedding_dim = 100
k = 0
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        k += 1
        embedding_matrix[i] = embedding_vector

Found 400000 word vectors.


In [121]:
from keras.layers import Bidirectional
model = Sequential()


model.add(Embedding(vocab_size + 1, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=True))


model.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))#,kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01)))
model.add(MaxPooling1D(3))
model.add(GlobalMaxPooling1D())
model.add(BatchNormalization())

model.add(Dense(64, activation='relu',kernel_regularizer=l2(0.17),bias_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(32, activation='sigmoid',kernel_regularizer=l2(0.19),bias_regularizer=l2(0.01)))
model.add(Dropout(0.3))
#model.add(Dense(16, kernel_regularizer=l2(0.001), activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 492, 100)          7654100   
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 492, 128)          38528     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 164, 128)          0         
_________________________________________________________________
global_max_pooling1d_7 (Glob (None, 128)               0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 128)               512       
_________________________________________________________________
dense_19 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_19 (Dropout)         (None, 64)               

In [122]:
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
history = model.fit(processed_x_train,y_train,validation_data=(processed_x_test,y_test),epochs=1,batch_size=32,verbose=1)



Train on 50000 samples, validate on 50000 samples
Epoch 1/1
50000/50000 [==============================] - 377s 8ms/step - loss: 1.0807 - accuracy: 0.9469 - val_loss: 0.2053 - val_accuracy: 0.9479


In [123]:
res=(model.predict_classes(processed_x_test))
#res=np.argmax(res)
from sklearn.metrics import classification_report
print(classification_report(res,y_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.95      0.97     50000

    accuracy                           0.95     50000
   macro avg       0.50      0.47      0.49     50000
weighted avg       1.00      0.95      0.97     50000



In [ ]:

import pymysql as mdb 
def update(result, ids):
    mydb = mdb.connect(
      host="127.0.0.1",
      user="root",
      passwd="",
      database="crm"
    )
    mycursor = mydb.cursor()
    sql = "UPDATE spam SET predict = %s WHERE id = %s"
    val = (result, ids)
    mycursor.execute(sql, val)
    mydb.commit()
    mydb.close()
while True:
    mydb = mdb.connect(
          host="127.0.0.1",
          user="root",
          passwd="",
          database="crm"
        )
    mycursor = mydb.cursor()
    sql = "SELECT review,predict,id FROM spam"
    mycursor.execute(sql)
   
    myresult = mycursor.fetchall()
    mydb.close()
    for x in myresult:
      ids= str(x[2])
      pic=str(x[0])
      fl=str(x[1])
      
      if fl=='':
        print('prediction')
        a=[]
        pic=str(pro(pic))
        print(pic)
        print('----------------------')
        print(pic[0:14])
        c=str(pic[0:14])
        print(c.split(' '))
        try:
            a,b,c,d,e=(c.split(' '))
        except:
            a,b,c,d,e,f=(c.split(' '))
        pic=int(e[0])
        print(e)
        print('----------------------')
        
        
        
        try:
            pic1=text_to_wordlist(pic)
            pic1=tokenizer.texts_to_sequences(pic)

            pic1=pad_sequences(pic, maxlen=max_len, padding='post', truncating='post')
            #print(pic)
            a.append(pic)
            res=(model.predict_classes(a))
            res=np.argmax(res)
            res=pic
            
        except:
            res=pic
            vb=0
        res=str(res)
        print(res)
        
        update(res, ids)    
        

          
